In [1]:
#load dependencies
import shadie
import numpy as np
import pyslim
import toyplot
import tskit
import toytree
import pandas as pd
import os
from toytree.utils.src.style_axes import set_axes_ticks_external as set_axes_ticks_external
from toytree.utils.src.style_axes import set_axes_box_outline as set_axes_box_outline
import toyplot.svg

## Save Data from `.trees` files as `.csv` files

In [ ]:
def save_data_het(dirpath, mutrate, chrom, simtype="vittaria"):
    """
    Takes `dirpath` to a directory with all the replicate sims with same params.   
    Saves three .csv files in an existing directory called "data/":
    - data/ex_heterozygosity.csv: expected heterozygosity
    - data/ob_heterozygosity.csv: observed heterozygosity
    
    dirpath: string relative path to directory with replicate sims
    mutrate: string denoting mutation rate
    chrom: string denoting chromosome (selection) type for sims
    simtype: string; default value = "vittaria"; changes sample times
    """
    
    ex_df = pd.DataFrame(columns = ['mut_rate', 'chrom', 'time', "mean_heterozygosity", "CI_5%", "CI_95%", "stdev"])
    ob_df = pd.DataFrame(columns = ['mut_rate', 'chrom', 'time', "mean_heterozygosity", "CI_5%", "CI_95%", "stdev"])
    theta_df = pd.DataFrame(columns = ['mut_rate', 'chrom', 'time', "mean_heterozygosity", "CI_5%", "CI_95%", "stdev"])
    
    #initialize the dictionary
    directory = dirpath
    
    if simtype == "vittaria":
        #for vittaria
        times = []
        for i in range(1,201):
            time = i*200
            times.append(time)
    
    else:
        #for homosporous fern contrast sims
        times = []
        for i in range(1,200):
            time = i*200
            times.append(time+1)
    
    final_ex_dict = {int(x): [] for x in times}
    final_ob_dict = {int(x): [] for x in times}
    final_theta_dict = {int(x): [] for x in times}
    
    #add data to the dictionary
    for filename in os.listdir(directory):
        #generate the full path
        f = os.path.join(directory, filename)
        
        # check if it is a file
        if os.path.isfile(f):
            #load the file
            ts = tskit.load(f)
            
            het_dict = ts.metadata['SLiM']['user_metadata']['ex_heterozygosity'][0]
            int_dict = {int(k):v for k,v in het_dict.items()}
            sorted_dict = {key:int_dict[key] for key in sorted(int_dict)}
            
            for key, value in sorted_dict.items():
                final_ex_dict[key].append(float(1000000*value[0]))
                
            ob_dict = ts.metadata['SLiM']['user_metadata']['ob_heterozygosity'][0]
            ob_int_dict = {int(k):v for k,v in ob_dict.items()}
            ob_sorted_dict = {key:ob_int_dict[key] for key in sorted(ob_int_dict)}
            
            for key, value in ob_sorted_dict.items():
                final_ob_dict[key].append(float(1000000*value[0]))
                
            theta_dict = ts.metadata['SLiM']['user_metadata']['theta'][0]
            theta_int_dict = {int(k):v for k,v in theta_dict.items()}
            theta_sorted_dict = {key:theta_int_dict[key] for key in sorted(theta_int_dict)}
            
            for key, value in theta_sorted_dict.items():
                final_theta_dict[key].append(float(1000000*value[0]))

    for key, value in final_ex_dict.items():
        mean_val = np.mean(value)
        low, high = scipy.stats.t.interval(confidence=0.95,
            df=len(value)-1,
            loc=mean_val,
            scale=scipy.stats.sem(value),
        )
        std = np.std(value)
    
        ex_df = pd.concat([pd.DataFrame([[mut_rate, chrom, key, mean_val, low, high, std]], columns=ex_df.columns), ex_df], 
                   ignore_index=True)
    
    #save the df
    ex_file = dirpath + "data/ex_heterozygosity.csv"
    
    ex_df.to_csv(ex_file, header = True, index = False)
    
    for key, value in final_ob_dict.items():
        mean_val = np.mean(value)
        low, high = scipy.stats.t.interval(
            confidence=0.95, 
            df=len(value)-1, 
            loc=mean_val, 
            scale=scipy.stats.sem(value),
        )
        std = np.std(value)
    
        ob_df = pd.concat([pd.DataFrame([[mut_rate, chrom, key, mean_val, low, high, std]], columns=ob_df.columns), ob_df], 
                   ignore_index=True)
    
    #save the df
    ob_file = dirpath + "data/ob_heterozygosity.csv"
    
    ob_df.to_csv(ob_file, header = True, index = False)
    
    for key, value in final_theta_dict.items():
        mean_val = np.mean(value)
        low, high = scipy.stats.t.interval(
            confidence=0.95, 
            df=len(value)-1, 
            loc=mean_val,
            scale=scipy.stats.sem(value),
        )
        std = np.std(value)
    
        theta_df = pd.concat([pd.DataFrame([[mut_rate, chrom, key, mean_val, low, high, std]], columns=theta_df.columns), theta_df], 
                   ignore_index=True)
    
    #save the df
    theta_file = dirpath + "data/theta.csv"
    
    theta_df.to_csv(theta_file, header = True, index = False)

In [ ]:
#do this for every set of sims; 
#all replicate .trees files should be saved in the same directory

save_data_het(dirpath="dirpath/trees/", mut_rate="low", chrom="neut", simtype="vittaria")

## Load the Data
### Purifying Selection

In [ ]:
pur_low_ex = pd.read_csv("data/purifying/lowmut/ex_heterozygosity.csv")
pur_low_ob = pd.read_csv("data/purifying/lowmut/ob_heterozygosity.csv")
pur_low_theta = pd.read_csv("data/purifying/lowmut/theta.csv")

hom_pur_low_ex = pd.read_csv("data/homosporous/purifying/lowmut/ex_heterozygosity.csv")
hom_pur_low_ob = pd.read_csv("data/homosporous/purifying/lowmut/ob_heterozygosity.csv")
hom_pur_low_theta = pd.read_csv("data/homosporous/purifying/lowmut/theta.csv")

In [ ]:
pur_med_ex = pd.read_csv("data/purifying/medmut/ex_heterozygosity.csv")
pur_med_ob = pd.read_csv("data/purifying/medmut/ob_heterozygosity.csv")
pur_med_theta = pd.read_csv("data/purifying/medmut/theta.csv")

hom_pur_med_ex = pd.read_csv("data/homosporous/purifying/medmut/ex_heterozygosity.csv")
hom_pur_med_ob = pd.read_csv("data/homosporous/purifying/medmut/data/ob_heterozygosity.csv")
hom_pur_med_theta = pd.read_csv("data/homosporous/purifying/medmut/trees/data/theta.csv")

In [ ]:
pur_high_ex = pd.read_csv("data/purifying/highmut/ex_heterozygosity.csv")
pur_high_ob = pd.read_csv("data/purifying/highmut/ob_heterozygosity.csv")
pur_high_theta = pd.read_csv("data/purifying/highmut/data/theta.csv")

hom_pur_high_ex = pd.read_csv("data/purifying/highmut/ex_heterozygosity.csv")
hom_pur_high_ob = pd.read_csv("data/purifying/highmut/ob_heterozygosity.csv")
hom_pur_high_theta = pd.read_csv("data/purifying/highmut/trees/data/theta.csv")

### Positive Selection

In [ ]:
pos_low_ex = pd.read_csv("data/positive/lowmut/ex_heterozygosity.csv")
pos_low_ob = pd.read_csv("data/positive/lowmut/ob_heterozygosity.csv")
pos_low_theta = pd.read_csv("data/positive/lowmut/theta.csv")

hom_pos_low_ex = pd.read_csv("data/homosporous/positive/lowmut/ex_heterozygosity.csv")
hom_pos_low_ob = pd.read_csv("data/homosporous/positive/lowmut/ob_heterozygosity.csv")
hom_pos_low_theta = pd.read_csv("data/homosporous/positive/lowmut/theta.csv")

In [ ]:
pos_med_ex = pd.read_csv("data/positive/medmut/ex_heterozygosity.csv")
pos_med_ob = pd.read_csv("data/positive/medmut/ob_heterozygosity.csv")
pos_med_theta = pd.read_csv("data/positive/medmut/theta.csv")

hom_pos_med_ex = pd.read_csv("data/homosporous/positive/medmut/ex_heterozygosity.csv")
hom_pos_med_ob = pd.read_csv("data/homosporous/positive/medmut/ob_heterozygosity.csv")
hom_pos_med_theta = pd.read_csv("data/homosporous/positive/medmut/theta.csv")

In [ ]:
pos_high_ex = pd.read_csv("data/positive/highmut/ex_heterozygosity.csv")
pos_high_ob = pd.read_csv("data/positive/highmut/ob_heterozygosity.csv")
pos_high_theta = pd.read_csv("data/positive/highmut/theta.csv")

hom_pos_high_ex = pd.read_csv("data/homosporous/positive/highmut/ex_heterozygosity.csv")
hom_pos_high_ob = pd.read_csv("data/homosporous/positive/highmut/ob_heterozygosity.csv")
hom_pos_high_theta = pd.read_csv("data/homosporous/positive/highmut/theta.csv")

### Neutral Simulations

In [ ]:
neut_low_ex = pd.read_csv("data/neutral/lowmut/ex_heterozygosity.csv")
neut_low_ob = pd.read_csv("data/neutral/lowmut/ob_heterozygosity.csv")
neut_low_theta = pd.read_csv("data/neutral/lowmut/theta.csv")

hom_neut_low_ex = pd.read_csv("data/homosporous/neutral/lowmut/ex_heterozygosity.csv")
hom_neut_low_ob = pd.read_csv("data/homosporous/neutral/lowmut/data/ob_heterozygosity.csv")
hom_neut_low_theta = pd.read_csv("data/homosporous/neutral/lowmut/trees/data/theta.csv")

In [ ]:
neut_med_ex = pd.read_csv("data/neutral/medmut/ex_heterozygosity.csv")
neut_med_ob = pd.read_csv("data/neutral/medmut/ob_heterozygosity.csv")
neut_med_theta = pd.read_csv("data/neutral/medmut/theta.csv")

hom_neut_med_ex = pd.read_csv("data/homosporous/neutral/medmut/ex_heterozygosity.csv")
hom_neut_med_ob = pd.read_csv("data/homosporous/neutral/medmut/ob_heterozygosity.csv")
hom_neut_med_theta = pd.read_csv("data/homosporous/neutral/medmut/theta.csv")

In [ ]:
neut_high_ex = pd.read_csv("data/neutral/highmut/ex_heterozygosity.csv")
neut_high_ob = pd.read_csv("data/neutral/highmut/ob_heterozygosity.csv")
neut_high_theta = pd.read_csv("data/neutral/highmut/theta.csv")

hom_neut_high_ex = pd.read_csv("data/homosporous/neutral/highmut/ex_heterozygosity.csv")
hom_neut_high_ob = pd.read_csv("data/homosporous/neutral/highmut/ob_heterozygosity.csv")
hom_neut_high_theta = pd.read_csv("data/homosporous/neutral/highmut/theta.csv")

### Added Sexual Reproduction
#### Rare Sexual Reproduction
All sims with added sexual reproduction were run with medium mutation rate = 5e-9

In [ ]:
lowsex_pur_med_ex = pd.read_csv("data/sexual/purifying/lowsex/ex_heterozygosity.csv")
lowsex_pur_med_ob = pd.read_csv("data/sexual/purifying/lowsex/ob_heterozygosity.csv")
lowsex_pur_med_theta = pd.read_csv("data/sexual/purifying/lowsex/theta.csv")

lowsex_pos_med_ex = pd.read_csv("data/sexual/positive/lowsex/ex_heterozygosity.csv")
lowsex_pos_med_ob = pd.read_csv("data/sexual/positive/lowsex/ob_heterozygosity.csv")
lowsex_pos_med_theta = pd.read_csv("data/sexual/positive/lowsex/theta.csv")

lowsex_neut_med_ex = pd.read_csv("data/sexual/neutral/lowsex/ex_heterozygosity.csv")
lowsex_neut_med_ob = pd.read_csv("data/sexual/neutral/lowsex/ob_heterozygosity.csv")
lowsex_neut_med_theta = pd.read_csv("data/sexual/neutral/lowsex/data/theta.csv")

#### Medium Sexual Reproduction

In [ ]:
medsex_pur_med_ex = pd.read_csv("data/sexual/purifying/medsex/ex_heterozygosity.csv")
medsex_pur_med_ob = pd.read_csv("data/sexual/purifying/medsex/ob_heterozygosity.csv")
medsex_pur_med_theta = pd.read_csv("data/sexual/purifying/medsex/theta.csv")

medsex_pos_med_ex = pd.read_csv("data/sexual/positive/medsex/ex_heterozygosity.csv")
medsex_pos_med_ob = pd.read_csv("data/sexual/positive/medsex/ob_heterozygosity.csv")
medsex_pos_med_theta = pd.read_csv("data/sexual/positive/medsex/theta.csv")

medsex_neut_med_ex = pd.read_csv("data/sexual/neutral/medsex/ex_heterozygosity.csv")
medsex_neut_med_ob = pd.read_csv("data/sexual/neutral/medsex/ob_heterozygosity.csv")
medsex_neut_med_theta = pd.read_csv("data/sexual/neutral/medsex/theta.csv")

#### High Sexual Reproduction

In [11]:
highsex_pur_med_ex = pd.read_csv("data/sexual/purifying/highsex/ex_heterozygosity.csv")
highsex_pur_med_ob = pd.read_csv("data/sexual/purifying/highsex/ob_heterozygosity.csv")
highsex_pur_med_theta = pd.read_csv("data/sexual/purifying/highsex/theta.csv")

highsex_pos_med_ex = pd.read_csv("data/sexual/positive/highsex/ex_heterozygosity.csv")
highsex_pos_med_ob = pd.read_csv("data/sexual/positive/highsex/ob_heterozygosity.csv")
highsex_pos_med_theta = pd.read_csv("data/sexual/positive/highsex/theta.csv")

highsex_neut_med_ex = pd.read_csv("data/sexual/neutral/highsex/ex_heterozygosity.csv")
highsex_neut_med_ob = pd.read_csv("data/sexual/neutral/highsex/ob_heterozygosity.csv")
highsex_neut_med_theta = pd.read_csv("data/sexual/neutral/highsex/theta.csv")

SyntaxError: unterminated string literal (detected at line 9) (2059962293.py, line 9)

### Add Ne & FIS Calculations to Theta DFs
In this section I calculate Ne (based on `theta=4*Ne*mu`) and Fis (`Fis=(expected_heterozygosity-observed_heterozygosity)/expected_heterozygosity`) and add them to the loaded theta dataframe. 

This should have been included in the `save_data_het()` function, but since our datasets already exists before we calculated these two values and it takes a long time to generate those .csv files, I ended up doing it manually as below. **If you are running new sims I suggest adding this to the`save_data_het()` function to save yourself a lot of typing.** 

Another note: because the theta value was calculated in SLiM, we must use the mutation rate based on SLiM generations, which is 1/2 the mutation rate per life cycle. These values are defined below. 

In [8]:
# define SLiM mutation rates
LOWMUT = 5e-10/2
MEDMUT = 5e-9/2
HIGHMUT = 5e-8/2

In [7]:
#low mutation rate
pur_low_theta["Ne"] = 1e-6*pur_low_theta["mean_heterozygosity"]/(LOWMUT*4)
pos_low_theta["Ne"] = 1e-6*pos_low_theta["mean_heterozygosity"]/(LOWMUT*4)
neut_low_theta["Ne"] = 1e-6*neut_low_theta["mean_heterozygosity"]/(LOWMUT*4)

pur_low_theta["Fis"] = (pur_low_ex["mean_heterozygosity"]-pur_low_ob["mean_heterozygosity"])/pur_low_ex["mean_heterozygosity"]
pos_low_theta["Fis"] = (pos_low_ex["mean_heterozygosity"]-pos_low_ob["mean_heterozygosity"])/pos_low_ex["mean_heterozygosity"]
neut_low_theta["Fis"] = (neut_low_ex["mean_heterozygosity"]-neut_low_ob["mean_heterozygosity"])/neut_low_ex["mean_heterozygosity"]

NameError: name 'pur_low_theta' is not defined

In [ ]:
# medium mutation rate
pur_med_theta["Ne"] = 1e-6*pur_med_theta["mean_heterozygosity"]/(MEDMUT*4)
pos_med_theta["Ne"] = 1e-6*pos_med_theta["mean_heterozygosity"]/(MEDMUT*4)
neut_med_theta["Ne"] = 1e-6*neut_med_theta["mean_heterozygosity"]/(MEDMUT*4)

pur_med_theta["Fis"] = (pur_med_ex["mean_heterozygosity"]-pur_med_ob["mean_heterozygosity"])/pur_med_ex["mean_heterozygosity"]
pos_med_theta["Fis"] = (pos_med_ex["mean_heterozygosity"]-pos_med_ob["mean_heterozygosity"])/pos_med_ex["mean_heterozygosity"]
neut_med_theta["Fis"] = (neut_med_ex["mean_heterozygosity"]-neut_med_ob["mean_heterozygosity"])/neut_med_ex["mean_heterozygosity"]

#sexual simulations
lowsex_pur_med_theta["Ne"] = 1e-6*lowsex_pur_med_theta["mean_heterozygosity"]/(MEDMUT*4)
lowsex_pos_med_theta["Ne"] = 1e-6*lowsex_pos_med_theta["mean_heterozygosity"]/(MEDMUT*4)
lowsex_neut_med_theta["Ne"] = 1e-6*lowsex_neut_med_theta["mean_heterozygosity"]/(MEDMUT*4)
medsex_pur_med_theta["Ne"] = 1e-6*medsex_pur_med_theta["mean_heterozygosity"]/(MEDMUT*4)
medsex_pos_med_theta["Ne"] = 1e-6*medsex_pos_med_theta["mean_heterozygosity"]/(MEDMUT*4)
medsex_neut_med_theta["Ne"] = 1e-6*medsex_neut_med_theta["mean_heterozygosity"]/(MEDMUT*4)
highsex_neut_med_theta["Ne"] = 1e-6*highsex_neut_med_theta["mean_heterozygosity"]/(MEDMUT*4)
highsex_pur_med_theta["Ne"] = 1e-6*highsex_pur_med_theta["mean_heterozygosity"]/(MEDMUT*4)
highsex_pos_med_theta["Ne"] = 1e-6*highsex_pos_med_theta["mean_heterozygosity"]/(MEDMUT*4)

lowsex_pur_med_theta["Fis"] = (lowsex_pur_med_ex["mean_heterozygosity"]-lowsex_pur_med_ob["mean_heterozygosity"])/lowsex_pur_med_ex["mean_heterozygosity"]
lowsex_pos_med_theta["Fis"] = (lowsex_pos_med_ex["mean_heterozygosity"]-lowsex_pos_med_ob["mean_heterozygosity"])/lowsex_pos_med_ex["mean_heterozygosity"]
lowsex_neut_med_theta["Fis"] = (lowsex_neut_med_ex["mean_heterozygosity"]-lowsex_neut_med_ob["mean_heterozygosity"])/lowsex_neut_med_ex["mean_heterozygosity"]
medsex_pur_med_theta["Fis"] = (medsex_pur_med_ex["mean_heterozygosity"]-medsex_pur_med_ob["mean_heterozygosity"])/medsex_pur_med_ex["mean_heterozygosity"]
medsex_pos_med_theta["Fis"] = (medsex_pos_med_ex["mean_heterozygosity"]-medsex_pos_med_ob["mean_heterozygosity"])/medsex_pos_med_ex["mean_heterozygosity"]
medsex_neut_med_theta["Fis"] = (medsex_neut_med_ex["mean_heterozygosity"]-medsex_neut_med_ob["mean_heterozygosity"])/medsex_neut_med_ex["mean_heterozygosity"]
highsex_neut_med_theta["Fis"] = (highsex_neut_med_ex["mean_heterozygosity"]-highsex_neut_med_ob["mean_heterozygosity"])/highsex_neut_med_ex["mean_heterozygosity"]
highsex_pur_med_theta["Fis"] = (highsex_pur_med_ex["mean_heterozygosity"]-highsex_pur_med_ob["mean_heterozygosity"])/highsex_pur_med_ex["mean_heterozygosity"]
highsex_pos_med_theta["Fis"] = (highsex_pos_med_ex["mean_heterozygosity"]-highsex_pos_med_ob["mean_heterozygosity"])/highsex_pos_med_ex["mean_heterozygosity"]

In [ ]:
# high mutation rate
pur_high_theta["Ne"] = 1e-6*pur_high_theta["mean_heterozygosity"]/(HIGHMUT*4)
pos_high_theta["Ne"] = 1e-6*pos_high_theta["mean_heterozygosity"]/(HIGHMUT*4)
neut_high_theta["Ne"] = 1e-6*neut_high_theta["mean_heterozygosity"]/(HIGHMUT*4)

pur_high_theta["Fis"] = (pur_high_ex["mean_heterozygosity"]-pur_high_ob["mean_heterozygosity"])/pur_high_ex["mean_heterozygosity"]
pos_high_theta["Fis"] = (pos_high_ex["mean_heterozygosity"]-pos_high_ob["mean_heterozygosity"])/pos_high_ex["mean_heterozygosity"]
neut_high_theta["Fis"] = (neut_high_ex["mean_heterozygosity"]-neut_high_ob["mean_heterozygosity"])/neut_high_ex["mean_heterozygosity"]

In [ ]:
# homosporous simulations
hom_pur_med_theta["Ne"] = 1e-6*hom_pur_med_theta["mean_heterozygosity"]/(MEDMUT*4)
hom_pos_med_theta["Ne"] = 1e-6*hom_pos_med_theta["mean_heterozygosity"]/(MEDMUT*4)
hom_neut_med_theta["Ne"] = 1e-6*hom_neut_med_theta["mean_heterozygosity"]/(MEDMUT*4)

hom_pur_med_theta["Fis"] = (hom_pur_med_ex["mean_heterozygosity"]-hom_pur_med_ob["mean_heterozygosity"])/hom_pur_med_ex["mean_heterozygosity"]
hom_pos_med_theta["Fis"] = (hom_pos_med_ex["mean_heterozygosity"]-hom_pos_med_ob["mean_heterozygosity"])/hom_pos_med_ex["mean_heterozygosity"]
hom_neut_med_theta["Fis"] = (hom_neut_med_ex["mean_heterozygosity"]-hom_neut_med_ob["mean_heterozygosity"])/hom_neut_med_ex["mean_heterozygosity"]


### Make the Data Lists for Graphing

In [9]:
data_lists_med = [[pur_med_theta, pur_med_ob, pur_med_ex], 
                  [pos_med_theta, pos_med_ob, pos_med_ex],
                  [neut_med_theta, neut_med_ob, neut_med_ex], 
                  [lowsex_pur_med_theta, lowsex_pur_med_ob, lowsex_pur_med_ex],
                  [lowsex_pos_med_theta, lowsex_pos_med_ob, lowsex_pos_med_ex],
                  [lowsex_neut_med_theta, lowsex_neut_med_ob, lowsex_neut_med_ex],
                  [medsex_pur_med_theta, medsex_pur_med_ob, medsex_pur_med_ex],
                  [medsex_pos_med_theta, medsex_pos_med_ob, medsex_pos_med_ex],
                  [medsex_neut_med_theta, medsex_neut_med_ob, medsex_neut_med_ex],
                  [highsex_neut_med_theta, highsex_neut_med_ob, highsex_neut_med_ex],
                  [highsex_pos_med_theta, highsex_pos_med_ob, highsex_pos_med_ex],
                  [highsex_pur_med_theta, highsex_pur_med_ob, highsex_pur_med_ex],
                 ]

NameError: name 'pur_med_theta' is not defined

In [10]:
control_lists = [[hom_pur_med_theta, hom_pur_med_ob, hom_pur_med_ex],
                      [hom_pos_med_theta, hom_pos_med_ob, hom_pos_med_ex],
                      [hom_neut_med_theta, hom_neut_med_ob, hom_neut_med_ex]]

NameError: name 'hom_pur_med_theta' is not defined

In [ ]:
data_list_theta = [pur_low_theta, pos_low_theta, neut_low_theta, pur_high_theta, pos_high_theta, neut_high_theta,
             pur_med_theta, pos_med_theta, neut_med_theta, lowsex_pur_med_theta, lowsex_pos_med_theta, 
             lowsex_neut_med_theta, medsex_pur_med_theta, medsex_pos_med_theta, medsex_neut_med_theta,
             highsex_neut_med_theta, highsex_pos_med_theta, highsex_pur_med_theta, hom_pur_med_theta, 
             hom_pos_med_theta, hom_neut_med_theta]

In [ ]:
data_list_ob = [pur_low_ob, pos_low_ob, neut_low_ob, pur_high_ob, pos_high_ob, neut_high_ob,
             pur_med_ob, pos_med_ob, neut_med_ob, lowsex_pur_med_ob, lowsex_pos_med_ob, 
             lowsex_neut_med_ob, medsex_pur_med_ob, medsex_pos_med_ob, medsex_neut_med_ob,
             highsex_neut_med_ob, highsex_pos_med_ob, highsex_pur_med_ob, hom_pur_med_ob, 
             hom_pos_med_ob, hom_neut_med_ob]

In [ ]:
data_list_ex = [pur_low_ex, pos_low_ex, neut_low_ex, pur_high_ex, pos_high_ex, neut_high_ex,
             pur_med_ex, pos_med_ex, neut_med_ex, lowsex_pur_med_ex, lowsex_pos_med_ex, 
             lowsex_neut_med_ex, medsex_pur_med_ex, medsex_pos_med_ex, medsex_neut_med_ex,
             highsex_neut_med_ex, highsex_pos_med_ex, highsex_pur_med_ex, hom_pur_med_ex, 
             hom_pos_med_ex, hom_neut_med_ex]

## Generate Table of Values at 10,000 gens
Takes two lists of datasets that have been loaded and the generation at which values will be recorded in SLiM generations (i.e. there are two SLiM generations for every 10,000 "full" life cycles, or true generations). 

In [ ]:
def get_data_at(data_lists, control_lists, time=20000):
    mut_rates = []
    selections = []
    reg_times = []
    thetas = []
    Nes = []
    Fiss = []
    Hos = []
    Hes = []
    
    reg_time = int((time)/2)
    
    for item in data_lists:
        df = item[0]
        theta = round(df.loc[df['time'] == time, 'mean_heterozygosity'].item(),3)*0.000001
        Ne = int(df.loc[df['time'] == time, 'Ne'].item())
        Fis = round(df.loc[df['time'] == time, 'Fis'].item(),3)
        mut_rate = df.loc[df['time'] == time, 'mut_rate'].item()
        selection = df.loc[df['time'] == time, 'chrom'].item()
        
        df1 = item[1]
        Ho = round(df1.loc[df['time'] == time, 'mean_heterozygosity'].item(),3)*0.000001
        
        df2 = item[2]
        He = round(df2.loc[df['time'] == time, 'mean_heterozygosity'].item(),3)*0.000001
        
        mut_rates.append(mut_rate)
        selections.append(selection)
        reg_times.append(reg_time)
        thetas.append(theta)
        Nes.append(Ne)
        Fiss.append(Fis)
        Hos.append(Ho)
        Hes.append(He)
    
    ctime = time+1
    for item in control_lists:
        df = item[0]
        theta = round(df.loc[df['time'] == ctime, 'mean_heterozygosity'].item(),3)*0.000001
        Ne = int(df.loc[df['time'] == ctime, 'Ne'].item())
        Fis = round(df.loc[df['time'] == ctime, 'Fis'].item(),3)
        mut_rate = df.loc[df['time'] == ctime, 'mut_rate'].item()
        selection = df.loc[df['time'] == ctime, 'chrom'].item()
        
        df1 = item[1]
        Ho = round(df1.loc[df['time'] == ctime, 'mean_heterozygosity'].item(),3)*0.000001
        
        df2 = item[2]
        He = round(df2.loc[df['time'] == ctime, 'mean_heterozygosity'].item(),3)*0.000001
        
        mut_rates.append(mut_rate)
        selections.append(selection)
        reg_times.append(reg_time)
        thetas.append(theta)
        Nes.append(Ne)
        Fiss.append(Fis)
        Hos.append(Ho)
        Hes.append(He)
        
    list_of_tuples = list(zip(mut_rates, selections, reg_times, Hes, Hos, thetas, Fiss, Nes))
    data_at_time = pd.DataFrame(list_of_tuples, columns = ['Mutation Rate', 'Selection', 'Generation', 
                                                 'He', 'Ho', 'Theta', 'Fis', 'Ne'])
    data_at_time.to_csv(f"data_at_{reg_time}.csv")

In [ ]:
get_data_at(data_lists_med, control_lists)

## Graphing
### Generate Colors using Color Brewer funtion in `toyplot`

In [3]:
colormap = toyplot.color.brewer.map("Spectral", domain_min=0, domain_max=1)
colormap

### Graphing Functions

In [4]:
def het_plot(datasets, error="CI", labels=[],
                        ymin = 0.0, 
                        ymax = 2000,
                        #xmin = 0, xmax = 0.3,
                        xlocations = [0, 4000, 8000, 12000, 16000, 20000],
                                     #24000, 28000, 32000, 36000, 40000], 
                        #ylocations=[0.0, 0.25, 0.50, 0.75, 0.9], #1.0
                        palette = "Set2",
                        plot_error = True,
                        title = "Heterozygosity",):
    xlabel = "Time (generations)"
    title = title
    
    #conf = int(100 - float(CIs[0][4:8]))

    #make the colors
    if isinstance(palette, str):
        palette = toyplot.color.brewer.palette(palette) #count=len(dataset)

    if error=="CI":
        error_str = "CI=95%"
    elif error=="std":
        error_str= "(σ)"
    
    label_style = {"text-anchor":"start", "-toyplot-anchor-shift":"5px", "font-size":"14px"}
    canvas = toyplot.Canvas(width=600, height=500)
    axes = canvas.cartesian(label = title, 
                            xlabel=xlabel, 
                            ylabel= f"Heterozygosity (\u00D710<sup>-6</sup>, {error_str})",
                            ymin = ymin, 
                            ymax = ymax,
                            #xmin = xmin, 
                            xmax = 20000,
                           )

    xlocations=xlocations
    #ylocations=ylocations
    axes.x.ticks.locator = toyplot.locator.Explicit(locations=xlocations) #format="{:.2f}"
    axes.x.ticks.labels.style = {"font-size":"13px"}
    axes.y.ticks.labels.style = {"font-size":"13px"}
    #axes.y.ticks.locator = toyplot.locator.Explicit(locations=ylocations)
    #axes.x.ticks.locator = toyplot.locator.Uniform(count=6)

    for x in range(len(datasets)):
        data = datasets[x]
        time = data['time'].tolist()

        mean_heterozygosity_str = data['mean_heterozygosity'].tolist()
        mean_heterozygosity=[float(i) for i in mean_heterozygosity_str]

        #format the error
        if error == "CI":
            for column in data.columns:
                if "CI_95%" in column:
                    error_plus = column
                if "CI_5%" in column:
                    error_minus = column
            CI_plus_str = data[error_plus].tolist()
            CI_plus=[float(i) for i in CI_plus_str]

            a = [mean_heterozygosity, CI_plus]
            y1 = list(map(sum,zip(*a)))

            CI_minus_str = data[error_minus].tolist()
            CI_minus=[float(i) for i in CI_minus_str]
            minus_CI = []
            for i in CI_minus:
                minus_CI.append(i*-1)

            b = [mean_heterozygosity, minus_CI]
            y2 = list(map(sum,zip(*b)))

        elif error == "std":
            prob_std_str = data['stdev'].tolist()
            prob_std=[float(i) for i in prob_std_str]

            a = [mean_heterozygosity, prob_std]
            y1 = list(map(sum,zip(*a)))

            minus_prob_std = []
            for i in prob_std:
                minus_prob_std.append(i*-1)

            b = [mean_heterozygosity, minus_prob_std]
            y2 = list(map(sum,zip(*b)))

        if plot_error == True:
            mark = axes.fill(time, y1, y2, opacity=0.2, color = palette[x])
        mark = axes.plot(time, mean_heterozygosity, color=palette[x])
        axes.text(16000, mean_heterozygosity[110], f"{labels[x]}", style=label_style, color = palette[x])

    #mark = axes.vlines(x=0.0)
    set_axes_ticks_external(axes)
    axes.label.style = {"font-size":"19px"}
    axes.x.label.style = {"font-size":"16px"}
    axes.y.label.style = {"font-size":"16px"}
    #set_axes_box_outline(axes)

    return canvas

In [5]:
def Ne_plot(datasets, error=None, labels=[],
                        ymin = 0.0, 
                        ymax = 1000,
                        #xmin = 0, xmax = 0.3,
                        xlocations = [0, 4000, 8000, 12000, 16000, 20000],
                                     #24000, 28000, 32000, 36000, 40000], 
                        #ylocations=[0.0, 0.25, 0.50, 0.75, 0.9], #1.0
                        palette = "Set2",
                        plot_error = True,
                        title = "Ne",):
    xlabel = "Time (generations)"
    title = title
    
    #conf = int(100 - float(CIs[0][4:8]))

    #make the colors
    if isinstance(palette, str):
        palette = toyplot.color.brewer.palette(palette) #count=len(dataset)

    if error=="CI":
        error_str = "CI=95%"
    elif error=="std":
        error_str= "(σ)"
    
    label_style = {"text-anchor":"start", "-toyplot-anchor-shift":"5px", "font-size":"14px"}
    canvas = toyplot.Canvas(width=600, height=500)
    axes = canvas.cartesian(label = title, 
                            xlabel=xlabel, 
                            ylabel= "Ne",
                            ymin = ymin, 
                            ymax = ymax,
                            #xmin = xmin, 
                            xmax = 20000,
                           )

    xlocations=xlocations
    #ylocations=ylocations
    axes.x.ticks.locator = toyplot.locator.Explicit(locations=xlocations) #format="{:.2f}"
    axes.x.ticks.labels.style = {"font-size":"13px"}
    axes.y.ticks.labels.style = {"font-size":"13px"}
    #axes.y.ticks.locator = toyplot.locator.Explicit(locations=ylocations)
    #axes.x.ticks.locator = toyplot.locator.Uniform(count=6)

    for x in range(len(datasets)):
        data = datasets[x]
        time = data['time'].tolist()

        Ne_str = data['Ne'].tolist()
        Ne=[float(i) for i in Ne_str]

        #format the error
        if error == "CI":
            for column in data.columns:
                if "CI_95%" in column:
                    error_plus = column
                if "CI_5%" in column:
                    error_minus = column
            CI_plus_str = data[error_plus].tolist()
            CI_plus=[float(i) for i in CI_plus_str]

            a = [mean_heterozygosity, CI_plus]
            y1 = list(map(sum,zip(*a)))

            CI_minus_str = data[error_minus].tolist()
            CI_minus=[float(i) for i in CI_minus_str]
            minus_CI = []
            for i in CI_minus:
                minus_CI.append(i*-1)

            b = [mean_heterozygosity, minus_CI]
            y2 = list(map(sum,zip(*b)))

        elif error == "std":
            prob_std_str = data['stdev'].tolist()
            prob_std=[float(i) for i in prob_std_str]

            a = [mean_heterozygosity, prob_std]
            y1 = list(map(sum,zip(*a)))

            minus_prob_std = []
            for i in prob_std:
                minus_prob_std.append(i*-1)

            b = [mean_heterozygosity, minus_prob_std]
            y2 = list(map(sum,zip(*b)))

        if plot_error == True:
            mark = axes.fill(time, y1, y2, opacity=0.2, color = palette[x])
        mark = axes.plot(time, Ne, color=palette[x])
        axes.text(16000, Ne[110], f"{labels[x]}", style=label_style, color = palette[x])

    #mark = axes.vlines(x=0.0)
    set_axes_ticks_external(axes)
    axes.label.style = {"font-size":"19px"}
    axes.x.label.style = {"font-size":"16px"}
    axes.y.label.style = {"font-size":"16px"}
    #set_axes_box_outline(axes)

    return canvas

In [ ]:
def Fis_plot(datasets, error=None, labels=[],
                        ymin = -1.0, 
                        ymax = 1.0,
                        #xmin = 0, xmax = 0.3,
                        xlocations = [0, 4000, 8000, 12000, 16000, 20000],
                                     #24000, 28000, 32000, 36000, 40000], 
                        #ylocations=[0.0, 0.25, 0.50, 0.75, 0.9], #1.0
                        palette = "Set2",
                        plot_error = True,
                        title = "Ne",):
    xlabel = "Time (generations)"
    title = title
    
    #conf = int(100 - float(CIs[0][4:8]))

    #make the colors
    if isinstance(palette, str):
        palette = toyplot.color.brewer.palette(palette) #count=len(dataset)

    if error=="CI":
        error_str = "CI=95%"
    elif error=="std":
        error_str= "(σ)"
    
    label_style = {"text-anchor":"start", "-toyplot-anchor-shift":"5px", "font-size":"14px"}
    canvas = toyplot.Canvas(width=600, height=500)
    axes = canvas.cartesian(label = title, 
                            xlabel=xlabel, 
                            ylabel= "Ne",
                            ymin = ymin, 
                            ymax = ymax,
                            #xmin = xmin, 
                            xmax = 20000,
                           )

    xlocations=xlocations
    #ylocations=ylocations
    axes.x.ticks.locator = toyplot.locator.Explicit(locations=xlocations) #format="{:.2f}"
    axes.x.ticks.labels.style = {"font-size":"13px"}
    axes.y.ticks.labels.style = {"font-size":"13px"}
    #axes.y.ticks.locator = toyplot.locator.Explicit(locations=ylocations)
    #axes.x.ticks.locator = toyplot.locator.Uniform(count=6)

    for x in range(len(datasets)):
        data = datasets[x]
        time = data['time'].tolist()

        Fis_str = data['Fis'].tolist()
        Fis=[float(i) for i in Fis_str]

        #format the error
        if error == "CI":
            for column in data.columns:
                if "CI_95%" in column:
                    error_plus = column
                if "CI_5%" in column:
                    error_minus = column
            CI_plus_str = data[error_plus].tolist()
            CI_plus=[float(i) for i in CI_plus_str]

            a = [mean_heterozygosity, CI_plus]
            y1 = list(map(sum,zip(*a)))

            CI_minus_str = data[error_minus].tolist()
            CI_minus=[float(i) for i in CI_minus_str]
            minus_CI = []
            for i in CI_minus:
                minus_CI.append(i*-1)

            b = [mean_heterozygosity, minus_CI]
            y2 = list(map(sum,zip(*b)))

        elif error == "std":
            prob_std_str = data['stdev'].tolist()
            prob_std=[float(i) for i in prob_std_str]

            a = [mean_heterozygosity, prob_std]
            y1 = list(map(sum,zip(*a)))

            minus_prob_std = []
            for i in prob_std:
                minus_prob_std.append(i*-1)

            b = [mean_heterozygosity, minus_prob_std]
            y2 = list(map(sum,zip(*b)))

        if plot_error == True:
            mark = axes.fill(time, y1, y2, opacity=0.2, color = palette[x])
        mark = axes.plot(time, Fis, color=palette[x])
        axes.text(16000, Fis[110], f"{labels[x]}", style=label_style, color = palette[x])

    #mark = axes.vlines(x=0.0)
    set_axes_ticks_external(axes)
    axes.label.style = {"font-size":"19px"}
    axes.x.label.style = {"font-size":"16px"}
    axes.y.label.style = {"font-size":"16px"}
    #set_axes_box_outline(axes)

    return canvas

In [ ]:
vittaria_het = het_plot([neut_med_ex, pur_med_ex, pos_med_ex, lowsex_neut_med_ex, lowsex_pur_med_ex, lowsex_pos_med_ex, 
         medsex_neut_med_ex, medsex_pur_med_ex, medsex_pos_med_ex,
         highsex_neut_med_ex, highsex_pur_med_ex, highsex_pos_med_ex], 
         plot_error = False,
         error = "std",
         ymax = 150,
         labels = ["Neutral", "Purifying", "Positive", "Low Sex, Neutral", "Low Sex, Purifying",
                  "Low Sex Postiive", "Med Sex Neutral", "Med Sex, Purifying", "Med Sex, Positive",
                  "High Sex, Neutral", "High Sex, Purifying", "High Sex, Positive"],
        title = "Expected Heterozygosity",
        palette = full)

In [ ]:
svg = toyplot.svg.rendervittaria_het, "../figs/vittaria_medmut_exhet_fullsexcomp.svg")

In [ ]:
vittaria_Ne = Ne_plot([hom_neut_med_theta, hom_pur_med_theta, hom_pos_med_theta, 
                    neut_med_theta, pur_med_theta, pos_med_theta, lowsex_neut_med_theta, 
                    lowsex_pur_med_theta, lowsex_pos_med_theta, 
                    medsex_neut_med_theta, medsex_pur_med_theta, medsex_pos_med_theta,
                    highsex_neut_med_theta, highsex_pur_med_theta, highsex_pos_med_theta], 
                    plot_error = False,
                    ymax = 2000,
         labels = ["Homosporous Neutral", "Homosporous Purifying", "Homosporous Positive",
                   "Neutral", "Purifying", "Positive", "Low Sex, Neutral", "Low Sex, Purifying",
                  "Low Sex Postiive", "Med Sex Neutral", "Med Sex, Purifying", "Med Sex, Positive",
                  "High Sex, Neutral", "High Sex, Purifying", "High Sex, Positive"],
        title = "Effective Population Size",
        palette = full2)

In [ ]:
svg = toyplot.svg.render(vittaria_Ne, "../figs/vittaria_Ne_control.svg")

In [ ]:
vittaria_Fis = Fis_plot([hom_neut_med_theta, hom_pur_med_theta, hom_pos_med_theta,
                    neut_med_theta, pur_med_theta, pos_med_theta, lowsex_neut_med_theta, 
                    lowsex_pur_med_theta, lowsex_pos_med_theta, 
                    medsex_neut_med_theta, medsex_pur_med_theta, medsex_pos_med_theta,
                    highsex_neut_med_theta, highsex_pur_med_theta, highsex_pos_med_theta], 
                    plot_error = False,
         labels = ["Homosporous Neutral", "Homosporous Purifying", "Homosporous Positive", 
                  "Neutral", "Purifying", "Positive", "Low Sex, Neutral", "Low Sex, Purifying",
                  "Low Sex Postiive", "Med Sex Neutral", "Med Sex, Purifying", "Med Sex, Positive",
                  "High Sex, Neutral", "High Sex, Purifying", "High Sex, Positive"],
        title = "Effective Population Size",
        palette = full2)

In [ ]:
svg = toyplot.svg.render(vittaria_Fis, "../figs/vittaria_Fis_control.svg")